In [1]:
import pandas as pd

In [33]:
sample = None

product = ['code', 'countries','url', 'product_name', 'brands', 'categories_en']

nutrient = ['energy_100g', 'fat_100g', 'carbohydrates_100g', 'fiber_100g', 'proteins_100g', 'polyols_100g', 'alcohol_100g']

columns_to_load = product + nutrient

opf_data = pd.read_csv('/Users/baslad01/data_dump/openfoodfacts/product_data/en.openfoodfacts.org.products.csv', sep='\t', encoding='utf-8', on_bad_lines='skip', nrows=sample,
                       usecols=columns_to_load)
opf_data.head()

/var/folders/_n/nybmwjg90rxfzcynss3bqkr80000gp/T/ipykernel_42280/2320493923.py:9: DtypeWarning: Columns (0,89,91) have mixed types. Specify dtype option on import or set low_memory=False.
  opf_data = pd.read_csv('/Users/baslad01/data_dump/openfoodfacts/product_data/en.openfoodfacts.org.products.csv', sep='\t', encoding='utf-8', on_bad_lines='skip', nrows=sample,


,code,url,product_name,brands,categories_en,countries,energy_100g,fat_100g,carbohydrates_100g,polyols_100g,fiber_100g,proteins_100g,alcohol_100g
0,54,http://world-en.openfoodfacts.org/product/0000...,Limonade artisanale a la rose,NaN,NaN,en:fr,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,63,http://world-en.openfoodfacts.org/product/0000...,Tablette Tanzanie,NaN,NaN,en:fr,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,http://world-en.openfoodfacts.org/product/0000...,CORDYCEPS,Cordyceps Sinensis,"Dietary supplements,Vitamins","Allemagne, États-Unis, en:fr",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,http://world-en.openfoodfacts.org/product/0000...,NaN,Curiositea House,it:olio-di-mandorle-dolci,Estados Unidos,293.0,4.0,3.00,NaN,NaN,1.00,NaN
4,3,http://world-en.openfoodfacts.org/product/0000...,Javvy protein coffee,Javvy,"Protein,Smoked-meat","Griechenland, Germany",816.0,10.73,14.63,NaN,0.98,0.51,NaN


In [30]:
# Remove the rows where the energy is missing
opf_data = opf_data[opf_data['energy_100g'].notna()]
# Remove energy if not float or int
opf_data = opf_data[opf_data['energy_100g'].apply(lambda x: isinstance(x, (int, float)))]
# Replace Nan with zeros on the nutrient columns
nutrient.remove('energy_100g')
for col in nutrient:
    opf_data[col] = opf_data[col].fillna(0)
opf_data.head()

,code,url,product_name,brands,categories_en,countries,energy_100g,fat_100g,carbohydrates_100g,polyols_100g,fiber_100g,proteins_100g,alcohol_100g
3,2,http://world-en.openfoodfacts.org/product/0000...,NaN,Curiositea House,it:olio-di-mandorle-dolci,Estados Unidos,293.0,4.00,3.00,0.0,0.00,1.00,0.0
4,3,http://world-en.openfoodfacts.org/product/0000...,Javvy protein coffee,Javvy,"Protein,Smoked-meat","Griechenland, Germany",816.0,10.73,14.63,0.0,0.98,0.51,0.0
5,4,http://world-en.openfoodfacts.org/product/0000...,Energy+ Focus,TruLabs,Sample,Německo,21800.0,0.00,18.20,0.0,0.00,0.00,0.0
6,5,http://world-en.openfoodfacts.org/product/0000...,Artikel 1,"EWL, bla","Plant-based foods and beverages,Plant-based fo...","Frankreich, Germany",840.0,0.00,8.00,0.0,88.00,0.00,0.0
8,7,http://world-en.openfoodfacts.org/product/0000...,Allumettes de bacon,Le marsigny,Butfalo-mac-and-cheese,en:Spain,519.0,5.00,0.80,0.0,0.00,19.00,0.0


In [43]:
# Create a linear regression model to predict the energy value given all the other nutrients in the product marked by _100g
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Filter the columns that have _100g in them
opf_num_features = opf_data.filter(regex='_100g')

# Convert all columns to float, setting errors='coerce' to convert non-convertible values to NaN
opf_num_features = opf_num_features.apply(pd.to_numeric, errors='coerce')

# Replace Nan with zeros
opf_num_features.fillna(0, inplace=True)

opf_num_features = opf_num_features.astype('float64')

In [44]:
# Remove the rows where the nutrient are above 100g
for col in nutrient:
    if col == 'energy_100g':
        continue
    opf_num_features = opf_num_features[opf_num_features[col] <= 100]


In [45]:
# Cap outliers for all the features
for col in opf_num_features.columns:
    q1 = opf_num_features[col].quantile(0.25)
    q3 = opf_num_features[col].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    opf_num_features[col] = opf_num_features[col].apply(lambda x: lower_bound if x < lower_bound else upper_bound if x > upper_bound else x)

# Split the data into train and test
X_train, X_test, y_train, y_test = train_test_split(opf_num_features.drop(columns=['energy_100g']), opf_num_features['energy_100g'], test_size=0.2, random_state=42)

# Create the model
model = LinearRegression(fit_intercept=False)

# Fit the model
model.fit(X_train, y_train)

# Predict the energy value
y_pred = model.predict(X_test)

# Evaluate the model
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, y_pred)

99087.03076634316

In [46]:
# Get the coefficients of the model
coefficients = model.coef_

# Get the intercept of the model
intercept = model.intercept_

# Print the formula with the intercept
formula = f'{round(intercept, 2)} + '
for i, col in enumerate(X_train.columns):
    formula += f'{round(coefficients[i], 2)} * {col} + '
# Remove the last ' + ' from the formula
formula = formula[:-3]

print(f'Linear Regression Formula: {formula}')

Linear Regression Formula: 0.0 + 47.38 * fat_100g + 14.99 * carbohydrates_100g + -0.0 * polyols_100g + 15.21 * fiber_100g + 14.81 * proteins_100g + 0.0 * alcohol_100g


## The theorethical model should be:

According to this link: https://mirabelle.openfoodfacts.org/products/energy_calculator

calories = 37 * fat + 17 * proteins + 29 * alcohol + 17 * (carbohydrates - polyols) + 10 * polyols + 8 * fiber

In [ ]:
# Get the intercept of the model